You work at the oil extraction company OilyGiant. Your task is to find the best places to open 200 new oil wells.

To complete this task, you will have to perform the following steps:
 
- Read the files with the parameters collected from oil wells in the selected region: crude oil quality and reserve volume.
- Create a model to predict the volume of reserves in new wells.
- Choose the oil wells that have the highest estimated values.
- Choose the region with the highest total profit for the selected oil wells.
- You have data on crude oil samples from three regions. The parameters of each oil well in the region are already known. Create a model that helps choose the region with the highest profit margin. Analyze potential benefits and risks using the bootstrapping technique.

## Importing libraries

In [20]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np


## Importing datasets

In [7]:
region1 = pd.read_csv('D:/Tripleten/datasets/geo_data_0.csv')
region2 = pd.read_csv('D:/Tripleten/datasets/geo_data_1.csv')
region3 = pd.read_csv('D:/Tripleten/datasets/geo_data_2.csv')

In [24]:
region1

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
...,...,...,...,...,...
99995,DLsed,0.971957,0.370953,6.075346,110.744026
99996,QKivN,1.392429,-0.382606,1.273912,122.346843
99997,3rnvd,1.029585,0.018787,-1.348308,64.375443
99998,7kl59,0.998163,-0.528582,1.583869,74.040764


In [76]:
databases = [region1,region2, region3]
rmse_regions = [] 
predictions = []
y = []

def model_training( databases ):
    
    for region in databases:
        target = region['product']
        features = region[['f0','f1','f2']]

        x_train, y_train, x_test, y_test = train_test_split(features,target, test_size=0.25, random_state=1234)

        model = LinearRegression()
        model.fit(x_train, x_test)
        prediction_valid =model.predict(y_train)
        mse = mean_squared_error(y_test, prediction_valid )
        rmse = np.sqrt(mse)
        # print(rmse)

        predictions.append(prediction_valid)
        y.append(y_test)
        rmse_regions.append([rmse])

model_training(databases)

print(rmse_regions)

[[37.562394183637785], [0.8939344738117692], [40.10318099258418]]


In [79]:

pd.DataFrame(predictions[0], columns=['pred_prod']).values
# pd.DataFrame(y[0]).values


array([[ 82.78094476],
       [ 41.89174099],
       [137.35610715],
       ...,
       [105.13914461],
       [ 90.9467246 ],
       [111.32639879]])